In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00395655632019043
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 36.67210850962243
200000 56.73134629635725
300000 50.690403652006566
400000 50.32865155656755
500000 56.25435788762203
600000 57.74331129186785
700000 56.89142938435921
800000 56.160999335977735
900000 57.52428938460673
1000000 45.02079771083375
1100000 37.11962370750815
1200000 62.06017110554745
1300000 57.016603978977805
1400000 60.58628704647183
1500000 63.90033976875775
1600000 58.96755320653095
1700000 59.93319109004364
1800000 52.29252616099637
1900000 66.24961018024516
2000000 48.304331222850124
2100000 30.748505880955605
2200000 57.606598047395
2300000 50.504606138462925
2400000 45.439636420014345
2500000 55.9659022063303
2600000 61.76544557457615
2700000 53.15435463852998
2800000 48.108700586567856
2900000 56.091333329462046
3000000 43.7748644917

30100000 79.11752843821675
30200000 75.54427097082028
30300000 32.67431230240962
30400000 74.43825188189058
30500000 73.47230784483334
30600000 74.1945349110618
30700000 74.1532408012643
30800000 69.1021182399174
30900000 69.67593234211581
31000000 53.55572952409564
31100000 40.653406486951745
31200000 68.05684489849992
31300000 68.09607030070167
31400000 71.72004107673881
31500000 72.2352081739148
31600000 71.40601567669836
31700000 74.89268447484912
31800000 41.73421105588321
31900000 67.72814502060507
32000000 72.66407038323057
32100000 76.47179765591495
32200000 72.98952933176786
32300000 35.833328067140755
32400000 73.25113153652681
32500000 75.6231650083877
32600000 74.40330164573912
32700000 72.63433864676914
32800000 70.76103647044795
32900000 68.7527168638304
33000000 56.47844008127452
33100000 40.75279332976976
33200000 70.18182825033594
33300000 69.20277837160955
33400000 72.59695634259835
33500000 73.39022930953111
33600000 74.4292144391678
33700000 75.14317809137577
338000

60600000 50.90148558526054
60700000 70.0181060099624
60800000 65.6316025197382
60900000 67.23337790568408
61000000 56.306475337697904
61100000 36.32799070788933
61200000 66.51703341953518
61300000 65.34885636219141
61400000 66.97445139092976
61500000 64.13231563134578
61600000 35.804384919571575
61700000 72.29686774575987
61800000 72.34627893610248
61900000 73.8600595327299
62000000 75.01966562869637
62100000 76.60677709563707
62200000 70.99145697148334
62300000 71.14282871601846
62400000 68.92799782591321
62500000 54.698970841196896
62600000 40.752496299100365
62700000 62.48866847273064
62800000 65.47127479078564
62900000 65.00203712606434
63000000 55.773667970252006
63100000 34.90435046248787
63200000 63.04445993749004
63300000 67.03159903532823
63400000 62.744627245028155
63500000 58.19126504689009
63600000 41.09808030255312
63700000 66.47145513154355
63800000 69.535308726472
63900000 69.0030033811809
64000000 77.72374869288086
64100000 76.01221172291946
64200000 73.81368928253428
6

91000000 55.03569805815601
91100000 31.386179644201952
91200000 59.7608250910913
91300000 49.32841628956872
91400000 43.024960603409205
91500000 63.342890001611465
91600000 60.792211048519185
91700000 62.492152869145855
91800000 67.60732643017737
91900000 68.39921114522275
92000000 67.04104371030124
92100000 67.60417287962238
92200000 69.08895906569921
92300000 65.32224431385674
92400000 70.56044302914805
92500000 60.4742334224918
92600000 66.5032000377047
92700000 61.824818898677876
92800000 28.935595668022213
92900000 63.7843456893176
93000000 53.82560878987486
93100000 30.585830249008467
93200000 61.18414346310272
93300000 42.461151164478046
93400000 48.90893367652329
93500000 61.30941293146635
93600000 60.62364156759287
93700000 67.28648443027805
93800000 67.54367577837215
93900000 68.80411124360646
94000000 69.5683764796036
94100000 70.81994340918727
94200000 67.8497868366988
94300000 64.1432922537649
94400000 64.52036580654034
94500000 67.05723169787312
94600000 61.23315894852020

120500000 75.95065162313213
120600000 76.93428439595102
120700000 75.879763099905
120800000 73.16428632008514
120900000 77.21512626574787
121000000 36.6272385200103
121100000 10.561002702601915
121200000 64.90165469111635
121300000 73.47709126653542
121400000 78.61010583787906
121500000 78.88594043049629
121600000 78.79594968217317
121700000 78.85882483419176
121800000 77.92242265063034
121900000 80.79234581754591
122000000 81.05977777102514
122100000 27.139818644372518
122200000 8.513650607864584
122300000 7.589771412012744
122400000 7.2770803715152805
122500000 6.619034056989675
122600000 6.750466745831984
122700000 6.67200104465574
122800000 6.270293786215623
122900000 6.325747521028173
123000000 6.257872164968605
123100000 6.226087827064104
123200000 6.225287991585245
123300000 6.582200428223597
123400000 6.648216028796872
123500000 6.19899555971027
123600000 6.633876656472729
123700000 7.007834325190019
123800000 6.419309951855869
123900000 6.4354537654284805
124000000 6.633157460

...

884300000 3.032657900558823
884400000 3.1068850833643835
884500000 2.172383817857951
884600000 3.395888168972564
884700000 2.845804330184536
884800000 3.5678575410080327
884900000 2.1632849493294595
885000000 3.169150164265301
885100000 2.6712711027531926
885200000 3.0361765964625116
885300000 3.1067882557581465
885400000 2.304935098536198
885500000 3.629894830893745
885600000 2.0712658173214895
885700000 3.4220059130869074
885800000 1.7038963144864043
885900000 3.5422937961226806
886000000 1.9222531830627443
886100000 2.9549696251827715
886200000 2.576183755415794
886300000 2.5958978948335125
886400000 4.371919958809807
886500000 2.440786081591486
886600000 1.6944230949324608
886700000 0.5260257933412015
886800000 0.5230463372839809
886900000 0.5191105634244051
887000000 0.5193188626303613
887100000 0.5194485348366876
887200000 0.5262497676062287
887300000 0.5463605242050846
887400000 0.675950032572139
887500000 0.674263359269364
887600000 0.5874826758866867
887700000 2.74255948505774

909000000 0.5206504569140242
909100000 0.515109456443327
909200000 0.3598302059823255
best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 1.0088872401072038
909400000 0.4672780761033257
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.35330713456404805
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.3270413426488625
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.29969463118473555
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.6202469803

920600000 1.2935677782951713
920700000 0.8659706438575361
920800000 1.3598451190238423
920900000 0.9720778691173176
921000000 1.2537139971778717
921100000 1.4311788825101766
921200000 0.8531421052515626
921300000 1.2884847450397001
921400000 0.8028920937405482
921500000 1.095225995713079
921600000 1.320923879039434
921700000 0.8814367086280698
921800000 1.3183625014353633
921900000 1.0632120126077813
922000000 1.378026054056537
922100000 0.41381409618297216
922200000 0.2573516437920312
922300000 0.24830921123945646
922400000 0.24083786499882143
922500000 0.22213577754080296
922600000 0.23139491954629418
922700000 0.24210374355401396
922800000 0.238136946580259
922900000 0.2467359634325213
923000000 0.23195983537922502
923100000 0.2294015515119731
923200000 0.2273256319767243
923300000 0.6855422282337348
923400000 1.0893586047169155
923500000 1.3790393685985889
923600000 1.3228474456351917
923700000 0.7964853664699727
923800000 1.160251877740636
923900000 0.7976102791679264
924000000 1.

944500000 0.05886704693331918
944600000 0.18488059775673152
944700000 0.2238339907181614
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.17214455960400632
944900000 0.12247836856605411
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.10684000062861575
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.09044908464146587
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.08964014869329731
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 

955400000 0.007903399886208773
955500000 0.004024593029686478
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.003447744051550494
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases o